In [ ]:
# Install dependencies (run once per environment)
%pip install -q dspy python-dotenv requests

# Mem0 stack (Docker) - run locally before using this notebook
# From this folder (contains docker-compose.yml):
# 1) Export your OpenAI key (same one used by notebooks):
#    export OPENAI_API_KEY=sk-...
# 2) Start services (API + Qdrant + Neo4j):
#    cd dspy/life_coach_agent
#    docker compose up -d
# 3) Verify the API is up:
#    curl -s http://localhost:8000/ | cat
# 4) Optional quick test (add + search):
#    curl -s -X POST http://localhost:8000/api/v1/memories \
#      -H "Content-Type: application/json" \
#      -d '{"user_id":"u_demo","memory":"Demo memory","enable_graph":true}' | cat
#    curl -s -X POST http://localhost:8000/api/v1/memories/search \
#      -H "Content-Type: application/json" \
#      -d '{"user_id":"u_demo","query":"demo","k":3,"enable_graph":true}' | cat
# 5) Stop services when done:
#    docker compose down
#


Note: you may need to restart the kernel to use updated packages.


In [20]:
# Basic imports and environment setup
import os
import dspy
from dotenv import load_dotenv

# Load API keys from .env (OPENAI_API_KEY is expected, already set in your env)
load_dotenv()

# Choose models similar to other notebooks
# Use the same style as blog-writer.ipynb
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)

# Configure DSPy default LM
dspy.configure(lm=lm)

print("DSPy configured. Life Coach agent setup starting...")



DSPy configured. Life Coach agent setup starting...


In [21]:
# Mem0 REST client utilities (mirror n8n HTTP nodes)
import json
import requests
from typing import Any, Dict, List, Optional

MEM0_BASE_URL = os.getenv("MEM0_BASE_URL", "http://localhost:8000")


def mem0_search(user_id: str, query: str, k: int = 3, enable_graph: bool = True, timeout: float = 8.0) -> Dict[str, Any]:
    """
    Call Mem0 search endpoint to retrieve relevant memories for a user.
    Mirrors the n8n HTTP Request node calling /api/v1/memories/search.
    """
    url = f"{MEM0_BASE_URL}/api/v1/memories/search"
    payload = {
        "user_id": user_id,
        "query": query,
        "k": k,
        "enable_graph": enable_graph,
    }
    headers = {"Content-Type": "application/json"}
    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        resp.raise_for_status()
        data = resp.json()
        # Normalize to a simple list of memory strings like the n8n code node
        results = data.get("results") or data.get("data") or []
        # If results are dicts with 'memory' field, extract them; else assume strings
        memory_texts = []
        if isinstance(results, list):
            for item in results[:10]:
                if isinstance(item, dict):
                    text = item.get("memory") or item.get("text") or ""
                    if text:
                        memory_texts.append(text)
                elif isinstance(item, str):
                    memory_texts.append(item)
        return {"memories": "\n".join(memory_texts), "raw": data}
    except Exception as e:
        return {"memories": "", "error": str(e)}


def mem0_add_memory(user_id: str, memory: str, enable_graph: bool = True, timeout: float = 8.0) -> Dict[str, Any]:
    """
    Call Mem0 add endpoint to store a new memory for a user.
    Mirrors the n8n HTTP Request node calling /api/v1/memories.
    """
    url = f"{MEM0_BASE_URL}/api/v1/memories"
    payload = {
        "user_id": user_id,
        "memory": memory,
        "enable_graph": enable_graph,
    }
    headers = {"Content-Type": "application/json"}
    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        resp.raise_for_status()
        return resp.json()
    except Exception as e:
        return {"error": str(e)}



In [22]:
# Short-term session memory (simple buffer like n8n Memory Buffer Window)
from collections import deque

class SessionMemory:
    """
    Minimal conversation buffer per session_id, with a fixed window size.
    Use this to feed the agent recent turns alongside Mem0 long-term memory.
    """
    def __init__(self, max_turns: int = 6):
        self.max_turns = max_turns
        self._store: dict[str, deque[str]] = {}

    def append(self, session_id: str, role: str, text: str) -> None:
        buf = self._store.setdefault(session_id, deque(maxlen=self.max_turns))
        buf.append(f"{role}: {text}")

    def get(self, session_id: str) -> list[str]:
        return list(self._store.get(session_id, deque()))

    def clear(self, session_id: str) -> None:
        self._store.pop(session_id, None)

session_memory = SessionMemory(max_turns=8)



In [23]:
# DSPy signatures and tools for the Life Coach agent
from typing import Any

# Global persona used across the module and demo
SYSTEM_PERSONA = (
    "You are a a highly sought-after executive coach and psychologist. "
    "Background includes mountain guide, BASE jumper, founder of One Day Coaching. "
    "Philosophy: transformations over incremental changes; presence, gratitude, reflection; "
    "self-leadership and resilience. Use frameworks: Circle of Control, Helicopter View, Time Jump, "
    "1–10 scale, Minimal Success Index. Communication: brief, sharp, practical; prefer 1–2 sentences; "
    "ask a powerful question if unclear; pick one actionable idea tied to the user's query."
)

class CoachSignature(dspy.Signature):
    """
    You are a highly sought-after executive coach and psychologist.
    Background: mountain guide, BASE jumper, founder of One Day Coaching.
    Philosophy: transformative change (mindset shifts), presence, self-leadership, resilience.
    Use frameworks: Circle of Control, Helicopter View, Time Jump, MSI, 1-10 scale,
    gratitude, reflection loop. Ask questions, be brief and practical.
    Communication:
    - Be brief, sharp, practical. Prefer 1–2 sentences.
    - Ask a powerful question if unclear how to help.
    - Pick one most relevant idea and make it actionable.

    Inputs:
    - user_id: caller identity for memory operations
    - session_id: short-term context key
    - user_input: the current message
    - history: conversation history across turns

    Output:
    - output: the coach reply
    """
    user_id: str = dspy.InputField()
    session_id: str = dspy.InputField()
    user_input: str = dspy.InputField()
    history: dspy.History = dspy.InputField()
    output: str = dspy.OutputField()


def tool_search_memories(user_id: str, query: str, k: int = 10) -> dict:
    """Search Mem0 for relevant memories for this user. Returns {tool, memories} (string)."""
    res = mem0_search(user_id=user_id, query=query, k=k, enable_graph=True)
    return {"tool": "search_memories", "memories": res.get("memories", ""), "raw": res}


def tool_add_memory(user_id: str, text: str) -> dict:
    """Add a raw user text as memory. Returns {tool, status}."""
    res = mem0_add_memory(user_id=user_id, memory=text, enable_graph=True)
    return {"tool": "add_memory", "status": "ok" if "error" not in res else "error", "raw": res}


def tool_get_session_context(session_id: str) -> dict:
    """Return recent turns from short-term session memory."""
    turns = session_memory.get(session_id)
    return {"tool": "session_context", "context": "\n".join(turns)}


def tool_append_session_turn(session_id: str, role: str, text: str) -> dict:
    """Append a turn to session buffer; returns count."""
    session_memory.append(session_id, role, text)
    return {"tool": "append_session", "count": len(session_memory.messages.get(session_id))}


react_agent = dspy.ReAct(
    CoachSignature,
    tools=[
        tool_search_memories,
        tool_add_memory,
        tool_get_session_context,
        tool_append_session_turn,
    ],
    max_iters=6,
)



In [24]:
# CoachAgentModule: manages dspy.History and session memory per turn
from dspy import History

# Define SYSTEM_PERSONA here to avoid cross-cell dependency
SYSTEM_PERSONA = (
    "You are a a highly sought-after executive coach and psychologist. "
    "Background includes mountain guide, BASE jumper, founder of One Day Coaching. "
    "Philosophy: transformations over incremental changes; presence, gratitude, reflection; "
    "self-leadership and resilience. Use frameworks: Circle of Control, Helicopter View, Time Jump, "
    "1–10 scale, Minimal Success Index. Communication: brief, sharp, practical; prefer 1–2 sentences; "
    "ask a powerful question if unclear; pick one actionable idea tied to the user's query."
)

class CoachAgentModule(dspy.Module):
    """
    A thin wrapper around the ReAct agent that:
    - Maintains a `dspy.History` across turns
    - Manages short-term session memory buffer
    - Calls Mem0 search and add via tools (through the ReAct agent)
    """
    def __init__(self):
        super().__init__()
        # Don't shadow dspy.Module.history (which DSPy uses internally as a list)
        self.conversation_history = History(messages=[])
        self.react = react_agent

    def forward(self, user_id: str, session_id: str, user_message: str) -> dict:
        # 1) Append user turn to short-term buffer now (like MemoryBufferWindow)
        session_memory.append(session_id, "user", user_message)

        # 2) Build composite prompt (persona + session context + memories + user text) via tools
        #    Use the ReAct tools inside the signature call; tools will be invoked by the agent.
        session_ctx = "\n".join(session_memory.get(session_id))

        # Query Mem0 directly for prompt enrichment (simple and explicit)
        mem_res = mem0_search(user_id=user_id, query=user_message, k=10)
        memories_text = mem_res.get("memories", "")

        composite = (
            f"SYSTEM:\n{SYSTEM_PERSONA}\n\n"
            f"CONTEXT (recent turns):\n{session_ctx}\n\n"
            f"MEMORIES:\n{memories_text}\n\n"
            f"USER:\n{user_message}"
        )

        # 3) Update dspy.History
        self.conversation_history.messages.append({"role": "user", "content": user_message})

        # 4) Run ReAct
        result = self.react(
            user_id=user_id,
            session_id=session_id,
            user_input=composite,
            history=self.conversation_history,
        )

        reply = result.output if hasattr(result, "output") else str(result)

        # 5) Update buffers and long-term memory
        session_memory.append(session_id, "assistant", reply)
        self.conversation_history.messages.append({"role": "assistant", "content": reply})
        _ = mem0_add_memory(user_id=user_id, memory=user_message, enable_graph=True)

        return {"reply": reply}



In [25]:
# Demo runner using CoachAgentModule (no __main__ guard)
import time

SYSTEM_PERSONA = (
    "You are a a highly sought-after executive coach and psychologist. "
    "Background includes mountain guide, BASE jumper, founder of One Day Coaching. "
    "Philosophy: transformations over incremental changes; presence, gratitude, reflection; "
    "self-leadership and resilience. Use frameworks: Circle of Control, Helicopter View, Time Jump, "
    "1–10 scale, Minimal Success Index. Communication: brief, sharp, practical; prefer 1–2 sentences; "
    "ask a powerful question if unclear; pick one actionable idea tied to the user's query."
)

agent = CoachAgentModule()

# Minimal quick test (adjust user/session as needed)
conv = [
    "I’m training for a 50k trail race but losing motivation after work.",
    "I also want to improve sleep without sacrificing morning runs.",
    "I felt anxious today about a missed workout.",
]
user_id = "u_alex"
session_id = "chat_1"

transcript = []
for msg in conv:
    res = agent(user_id=user_id, session_id=session_id, user_message=msg)
    transcript.append({"user": msg, "assistant": res["reply"]})
    time.sleep(0.2)

for turn in transcript:
    print("\nUser:", turn["user"]) 
    print("Coach:", turn["assistant"]) 




User: I’m training for a 50k trail race but losing motivation after work.
Coach: Do a 20-minute MSI run/walk after work—no pace targets, just start, be present, and finish the time as a non‑negotiable win. On a scale of 1–10, how committed are you to doing this tonight?

User: I also want to improve sleep without sacrificing morning runs.
Coach: Pick a fixed wake time, work backwards to allow 7.5–8 hours of sleep, and make a 45‑minute wind‑down (no screens last 30 min, dim lights, 10 min breathing/stretch) a non‑negotiable cue so morning runs stay possible. What time do you need to wake for your morning runs?

User: I felt anxious today about a missed workout.
Coach: Do a 20‑minute recovery now: 5 minutes of focused breathing, then 15 minutes easy walk/movement—make it a non‑negotiable reset to convert anxiety into calm. On a scale of 1–10, how committed are you to doing this in the next hour?
